In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2023-03-17 06:41:20.026627: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 06:41:20.325768: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glob/development-tools/versions/oneapi/2023.0.1/oneapi/vpl/2023.0.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/tbb/2021.8.0/env/../lib/intel64/gcc4.8:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/rkcommon/1.10.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray_studio/0.11.1/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray/2

In [3]:
# Set up the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_labels = pd.read_csv('Emergency_Vehicles/train.csv')
# Convert the column to strings
train_labels['emergency_or_not'] = train_labels['emergency_or_not'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory='Emergency_Vehicles/train',
    x_col='image_names',
    y_col='emergency_or_not',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')


Found 1380 validated image filenames belonging to 2 classes.


/home/u184510/.local/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 266 invalid image filename(s) in x_col="image_names". These filename(s) will be ignored.
  warnings.warn(


In [4]:
# Load the pre-trained ResNet-50 model and freeze the base layers
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

# Add new fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model with binary cross-entropy loss and Adam optimizer
model = Model(inputs=base_model.input, outputs=predictions)

#freeze Layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)

2023-03-17 06:41:30.307853: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glob/development-tools/versions/oneapi/2023.0.1/oneapi/vpl/2023.0.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/tbb/2021.8.0/env/../lib/intel64/gcc4.8:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/rkcommon/1.10.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray_studio/0.11.1/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray/2.10.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/openvkl/1.3.1/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/oidn/1.4.3/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/mpi/2021.8.0//libfabric/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/mpi/2021.8.0//lib/release:/glob/development-tools/versions/oneapi

Epoch 1/20
43/43 [==============================] - 84s 2s/step - loss: 0.6926 - accuracy: 0.5482
Epoch 2/20
43/43 [==============================] - 68s 2s/step - loss: 0.6725 - accuracy: 0.5645
Epoch 3/20
43/43 [==============================] - 69s 2s/step - loss: 0.6542 - accuracy: 0.6239
Epoch 4/20
43/43 [==============================] - 71s 2s/step - loss: 0.6363 - accuracy: 0.6491
Epoch 5/20
43/43 [==============================] - 67s 2s/step - loss: 0.6323 - accuracy: 0.6684
Epoch 6/20
43/43 [==============================] - 81s 2s/step - loss: 0.6209 - accuracy: 0.6911
Epoch 7/20
43/43 [==============================] - 81s 2s/step - loss: 0.6139 - accuracy: 0.6773
Epoch 8/20
43/43 [==============================] - 80s 2s/step - loss: 0.6081 - accuracy: 0.6818
Epoch 9/20
43/43 [==============================] - 57s 1s/step - loss: 0.6064 - accuracy: 0.6818
Epoch 10/20
43/43 [==============================] - 80s 2s/step - loss: 0.6023 - accuracy: 0.6803
Epoch 11/20
43/43 [

In [5]:
# Load the test CSV file with filenames
t_test_df = pd.read_csv("Emergency_Vehicles/train.csv")

In [6]:
# Create a generator for the test images
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="Emergency_Vehicles/train",
    x_col='image_names',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)

# Use the trained model to make predictions on the test data
preds = model.predict_generator(t_test_generator, steps=None)
# Remove any missing filenames from the test CSV file
missing_filenames = set(t_test_generator.filenames) - set(t_test_df['image_names'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['image_names'].isin(missing_filenames)]

Found 1380 validated image filenames.


/home/u184510/tmp/ipykernel_471002/1260252469.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(t_test_generator, steps=None)


In [7]:
li = []
for i in preds.ravel():
    if i>0.5:
        li.append(1)
    else:
        li.append(0)
     
# Create a DataFrame with the filenames and predictions
df = pd.DataFrame({'image_names': t_test_generator.filenames[:len(preds)], 'emergency_or_not': li})

test_inst_df=pd.merge(df, t_test_df, on='image_names', how='inner')

In [8]:
accuracy_score(df["emergency_or_not"],test_inst_df['emergency_or_not_y']) 

0.7275362318840579

In [9]:
confusion_matrix(df["emergency_or_not"],test_inst_df['emergency_or_not_y']) 

array([[640, 292],
       [ 84, 364]])

In [10]:
# Save the model
model.save('models/resnet.h5')